In [1]:
import sympy
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import symfem
from sympy import Rational
import dill
dill.settings["recurse"] = True
from sympy import Rational as R

In [2]:
x, y, x_1, y_1, x_2, y_2, x_3, y_3 = sympy.symbols("x y x_1 y_1 x_2 y_2 x_3 y_3")

rotation_matrix = sympy.Matrix(
    [
        [+R(0), +R(1)],
        [-R(1), +R(0)]
    ]
)

t1_hat = sympy.Matrix([-R(1), +R(1)]) / sympy.sqrt(2)
t2_hat = sympy.Matrix([+R(0), +R(1)])
t3_hat = sympy.Matrix([+R(1), +R(0)])

n1_hat = rotation_matrix @ t1_hat
n2_hat = rotation_matrix @ t2_hat
n3_hat = rotation_matrix @ t3_hat

### Get basis functions

In [3]:
monomial_basis = sympy.Matrix(
[1,
 x,
 x**2,
 x**3,
 x**4,
 x**5,
 y,
 x*y,
 x**2*y,
 x**3*y,
 x**4*y,
 y**2,
 x*y**2,
 x**2*y**2,
 x**3*y**2,
 y**3,
 x*y**3,
 x**2*y**3,
 y**4,
 x*y**4,
 y**5]
)

In [4]:
Vander = sympy.Matrix(np.zeros((21, 21)))

for i, basis in enumerate(monomial_basis):

    v1 = {x: 0, y: 0}
    Vander[i, 0] = basis.subs(v1)
    Vander[i, 1] = basis.diff(x).subs(v1)
    Vander[i, 2] = basis.diff(y).subs(v1)
    Vander[i, 3] = basis.diff(x, x).subs(v1)
    Vander[i, 4] = basis.diff(x, y).subs(v1)
    Vander[i, 5] = basis.diff(y, y).subs(v1)

    v2 = {x: 1, y: 0}
    Vander[i, 6] = basis.subs(v2)
    Vander[i, 7] = basis.diff(x).subs(v2)
    Vander[i, 8] = basis.diff(y).subs(v2)
    Vander[i, 9] = basis.diff(x, x).subs(v2)
    Vander[i, 10] = basis.diff(x, y).subs(v2)
    Vander[i, 11] = basis.diff(y, y).subs(v2)

    v3 = {x: 0, y: 1}
    Vander[i, 12] = basis.subs(v3)
    Vander[i, 13] = basis.diff(x).subs(v3)
    Vander[i, 14] = basis.diff(y).subs(v3)
    Vander[i, 15] = basis.diff(x, x).subs(v3)
    Vander[i, 16] = basis.diff(x, y).subs(v3)
    Vander[i, 17] = basis.diff(y, y).subs(v3)

    e1 = {x: R(1, 2), y: R(1, 2)}
    dn1 = n1_hat[0]*basis.diff(x) + n1_hat[1]*basis.diff(y)
    Vander[i, 18] = dn1.subs(e1)

    e2 = {x: 0, y: R(1, 2)}
    dn2 = n2_hat[0]*basis.diff(x) + n2_hat[1]*basis.diff(y)
    Vander[i, 19] = dn2.subs(e2)

    e3 = {x: R(1, 2), y: 0}
    dn3 = n3_hat[0]*basis.diff(x) + n3_hat[1]*basis.diff(y)
    Vander[i, 20] = dn3.subs(e3)

In [5]:
argyris_basis = Vander.inv() @ monomial_basis

In [17]:
dill.dump(
    sympy.lambdify(
        [x, y], 
        argyris_basis,
        ),
    open("../calculations/argyris_basis", "wb")
)

# Necessary symbols

In [9]:
G1_hat = n1_hat.row_join(t1_hat).T
G2_hat = n2_hat.row_join(t2_hat).T
G3_hat = n3_hat.row_join(t3_hat).T

l1 = ((x_3 - x_2) ** 2 + (y_3 - y_2) ** 2) ** R(1, 2)
l2 = ((x_3 - x_1) ** 2 + (y_3 - y_1) ** 2) ** R(1, 2)
l3 = ((x_2 - x_1) ** 2 + (y_2 - y_1) ** 2) ** R(1, 2)

t1 = sympy.Matrix([x_3 - x_2, y_3 - y_2]) / l1
t2 = sympy.Matrix([x_3 - x_1, y_3 - y_1]) / l2
t3 = sympy.Matrix([x_2 - x_1, y_2 - y_1]) / l3

n1 = rotation_matrix @ t1
n2 = rotation_matrix @ t2
n3 = rotation_matrix @ t3

tau_1 = sympy.Matrix([t1[0] ** 2, 2 * t1[0] * t1[1], t1[1] ** 2])
tau_2 = sympy.Matrix([t2[0] ** 2, 2 * t2[0] * t2[1], t2[1] ** 2])
tau_3 = sympy.Matrix([t3[0] ** 2, 2 * t3[0] * t3[1], t3[1] ** 2])

delta = x_1 * y_2 - x_1 * y_3 - x_2 * y_1 + x_2 * y_3 + x_3 * y_1 - x_3 * y_2

P_1_x = (-y_1 + y_3) / delta
P_1_y = (+x_1 - x_3) / delta

P_2_x = (+y_1 - y_2) / delta
P_2_y = (-x_1 + x_2) / delta

J = sympy.Matrix(
    [
        [P_1_x, P_1_y],
        [P_2_x, P_2_y],
    ]
)

THETA = sympy.Matrix(
    [
        [P_1_x**2, 2 * P_1_x * P_2_x, P_2_x**2],
        [P_1_y * P_1_x, P_1_y * P_2_x + P_1_x * P_2_y, P_2_x * P_2_y],
        [P_1_y**2, 2 * P_1_y * P_2_y, P_2_y**2],
    ]
)

G1 = n1.row_join(t1).T
G2 = n2.row_join(t2).T
G3 = n3.row_join(t3).T

B1 = G1_hat @ J.inv().T @ G1.T
B2 = G2_hat @ J.inv().T @ G2.T
B3 = G3_hat @ J.inv().T @ G3.T

right_vals_symbols = sympy.symbols("W1:25")
right_vals = sympy.Matrix(right_vals_symbols[:21])

right_vals[18] = n1[0] * right_vals_symbols[18] + n1[1] * right_vals_symbols[19]
right_vals[19] = n2[0] * right_vals_symbols[20] + n2[1] * right_vals_symbols[21]
right_vals[20] = n3[0] * right_vals_symbols[22] + n3[1] * right_vals_symbols[23]

### Calculate bilinear form and right values for forward pushed reference element

In [50]:
N = len(argyris_basis)

#This is done for fast integration
J11, J12, J21, J22 = sympy.symbols("J11, J12, J21, J22")
J_simp = sympy.Matrix([
    [J11, J12],
    [J21, J22]
])

result_b = [[0 for i in range(N)] for j in range(N)]
result_B = [[0 for i in range(N)] for j in range(N)]
result_A = [[0 for i in range(N)] for j in range(N)]

for idx, jdx in tqdm([(idx, jdx) for idx in range(N) for jdx in range(N)]):
    first = argyris_basis[idx]
    second = argyris_basis[jdx]

    # -----------------------U------------------------------------------------------------------------------
    u_x = first.diff(x) * J_simp.row(0)[0] + first.diff(y) * J_simp.row(1)[0]
    u_y = first.diff(x) * J_simp.row(0)[1] + first.diff(y) * J_simp.row(1)[1]

    u_xx = u_x.diff(x) * J_simp.row(0)[0] + u_x.diff(y) * J_simp.row(1)[0]
    u_xy = u_x.diff(x) * J_simp.row(0)[1] + u_x.diff(y) * J_simp.row(1)[1]

    u_yy = u_y.diff(x) * J_simp.row(0)[1] + u_y.diff(y) * J_simp.row(1)[1]
    # ------------------------------------------------------------------------------------------------------

    # -----------------------V------------------------------------------------------------------------------
    v_x = second.diff(x) * J_simp.row(0)[0] + second.diff(y) * J_simp.row(1)[0]
    v_y = second.diff(x) * J_simp.row(0)[1] + second.diff(y) * J_simp.row(1)[1]

    v_xx = v_x.diff(x) * J_simp.row(0)[0] + v_x.diff(y) * J_simp.row(1)[0]
    v_xy = v_x.diff(x) * J_simp.row(0)[1] + v_x.diff(y) * J_simp.row(1)[1]

    v_yy = v_y.diff(x) * J_simp.row(0)[1] + v_y.diff(y) * J_simp.row(1)[1]
    # ------------------------------------------------------------------------------------------------------

    # -----------------------bilinear-form------------------------------------------------------------------
    A = u_xx * v_xx
    B = u_xy * v_xy
    b = u_xx * v_yy
    C = u_yy * v_yy
    # ------------------------------------------------------------------------------------------------------

    # -----------------------integrals----------------------------------------------------------------------
    sym_int_B = sympy.integrate(A + 2 * B + C, (y, 0, 1 - x), (x, 0, 1))
    sym_int_b = sympy.integrate(A + 2 * b + C, (y, 0, 1 - x), (x, 0, 1))
    sym_int_A = sympy.integrate(first * second, (y, 0, 1 - x), (x, 0, 1))

    result_B[idx][jdx] = sym_int_B
    result_b[idx][jdx] = sym_int_b
    result_A[idx][jdx] = sym_int_A
# -----------------------------------------------------------------------------------------------------

result_b = sympy.Matrix(result_b)
result_B = sympy.Matrix(result_B)
result_A = sympy.Matrix(result_A)

result_A = result_A * abs(J.inv().det())
result_b = result_b * abs(J.inv().det())
result_B = result_B * abs(J.inv().det())

100%|██████████| 441/441 [55:02<00:00,  7.49s/it] 


In [51]:
N = len(argyris_basis)
result_C = [0 for j in range(N)]

for idx in tqdm(range(N)):
    u = argyris_basis[idx]
    sym_int_C = sympy.integrate(u, (y, 0, 1 - x), (x, 0, 1))
    result_C[idx] = sym_int_C

result_C = sympy.Matrix(result_C)
result_C = result_C * abs(J.inv().det())

100%|██████████| 21/21 [00:02<00:00,  7.46it/s]


In [52]:
result_b = result_b.subs({
        J11: J[0, 0],
        J12: J[0, 1],
        J21: J[1, 0],
        J22: J[1, 1]
})

result_B = result_B.subs({
        J11: J[0, 0],
        J12: J[0, 1],
        J21: J[1, 0],
        J22: J[1, 1]
})

### Save the results

In [53]:
dill.dump(result_b, open("../calculations/pushed_forward_result_for_b", "wb"))
dill.dump(result_B, open("../calculations/pushed_forward_result_for_B", "wb"))
dill.dump(result_A, open("../calculations/pushed_forward_result_for_A", "wb"))
dill.dump(result_C, open("../calculations/pushed_forward_result_for_C", "wb"))

### Load the results

In [54]:
result_b = dill.load(open("../calculations/pushed_forward_result_for_b", "rb"))
result_B = dill.load(open("../calculations/pushed_forward_result_for_B", "rb"))
result_A = dill.load(open("../calculations/pushed_forward_result_for_A", "rb"))
result_C = dill.load(open("../calculations/pushed_forward_result_for_C", "rb"))

# Define transformation matrices

### $V^{c}$

In [10]:
THETA_s = sympy.simplify(THETA * delta**2)
THETA_inv = THETA_s.inv() * delta**2

In [11]:
V_c = sympy.diag(
    1,
    J.inv().T,
    THETA_inv,
    1,
    J.inv().T,
    THETA_inv,
    1,
    J.inv().T,
    THETA_inv,
    B1,
    B2,
    B3,
)

### $E$

In [12]:
E = np.zeros((21, 24), dtype=int)
for i in range(21):
    for j in range(24):
        if (i <= 18) and (j <= 18) and (i == j):
            E[i][j] = 1
        elif (i == 19) and (j == 20):
            E[i][j] = 1
        elif (i == 20) and (j == 22):
            E[i][j] = 1
E = sympy.Matrix(E)

### $D$

In [13]:
D = sympy.Matrix(
    [
        [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
        [
            0,
            0,
            0,
            0,
            0,
            0,
            -R(15, 8) / l1,
            -R(7, 16) * t1[0],
            -R(7, 16) * t1[1],
            -l1 / 32 * tau_1[0],
            -l1 / 32 * tau_1[1],
            -l1 / 32 * tau_1[2],
            +R(15, 8) / l1,
            -R(7, 16) * t1[0],
            -R(7, 16) * t1[1],
            +l1 / 32 * tau_1[0],
            +l1 / 32 * tau_1[1],
            +l1 / 32 * tau_1[2],
            0,
            0,
            0,
        ],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
        [
            -R(15, 8) / l2,
            -R(7, 16) * t2[0],
            -R(7, 16) * t2[1],
            -l2 / 32 * tau_2[0],
            -l2 / 32 * tau_2[1],
            -l2 / 32 * tau_2[2],
            0,
            0,
            0,
            0,
            0,
            0,
            +R(15, 8) / l2,
            -R(7, 16) * t2[0],
            -R(7, 16) * t2[1],
            +l2 / 32 * tau_2[0],
            +l2 / 32 * tau_2[1],
            +l2 / 32 * tau_2[2],
            0,
            0,
            0,
        ],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
        [
            -R(15, 8) / l3,
            -R(7, 16) * t3[0],
            -R(7, 16) * t3[1],
            -l3 / 32 * tau_3[0],
            -l3 / 32 * tau_3[1],
            -l3 / 32 * tau_3[2],
            +R(15, 8) / l3,
            -R(7, 16) * t3[0],
            -R(7, 16) * t3[1],
            +l3 / 32 * tau_3[0],
            +l3 / 32 * tau_3[1],
            +l3 / 32 * tau_3[2],
            0,
            0,
            0,
            0,
            0,
            0,
            0,
            0,
            0,
        ],
    ]
)

### $M$

In [14]:
V = E @ V_c @ D
M = V.T

In [16]:
dill.dump(
    sympy.lambdify(
        [x_1, y_1, x_2, y_2, x_3, y_3], 
        M
        ),
    open("../calculations/argyris_M", "wb")
)

### Apply transformations

In [60]:
bilinear_form_biharmonic_without_mixed = M @ result_b @ M.T

In [61]:
bilinear_form_biharmonic_with_mixed = M @ result_B @ M.T

In [62]:
right_part = M @ result_A @ M.T @ right_vals

In [63]:
integral_values = M @ result_C

In [64]:
final_parameters = [x_1, y_1, x_2, y_2, x_3, y_3]

In [65]:
bilinear_form_biharmonic_without_mixed_f = sympy.lambdify(
    final_parameters, 
    bilinear_form_biharmonic_without_mixed,
    cse=True
)

In [66]:
bilinear_form_biharmonic_with_mixed_f = sympy.lambdify(
    final_parameters, 
    bilinear_form_biharmonic_with_mixed,
    cse=True
)

In [67]:
right_part_f = sympy.lambdify(
    [*final_parameters, *right_vals_symbols],
    right_part, 
    cse=True
)

In [68]:
integral_values_f = sympy.lambdify(
    final_parameters,
    integral_values, 
    cse=True
)

### Save the results

In [70]:
dill.dump(
    bilinear_form_biharmonic_without_mixed_f,
    open("../calculations/argyris_quintic_biharmonic_matrix_without_mixed_simplified", "wb"),
)
dill.dump(
    bilinear_form_biharmonic_with_mixed_f,
    open("../calculations/argyris_quintic_biharmonic_matrix_with_mixed_simplified", "wb"),
)
dill.dump(
    right_part_f,
    open("../calculations/argyris_quintic_biharmonic_matrix_right_values_simplified", "wb"),
)
dill.dump(
    integral_values_f,
    open("../calculations/argyris_quintic_biharmonic_matrix_integral_values_simplified", "wb"),
)